Import a couple of low-level packages(i.e. no scikit learn or pandas or numpy or anything fancy) for processing.

In [1]:
import csv
import random
import math
import re
import string

Load in the csv file, add each line to a list and adjust for header (cut it off) if relevant. 

In [2]:
def loadCsv(filename, header=True):
    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        dataset = []
        for row in csvReader:
            dataset.append(row)
    if header: dataset = dataset[1:]
    return dataset

Divide the dataset into two different lists: data (the tweet, in this case) and target variables. Convert the tags 'sexism' and 'racism' into one tag representing hate speech, 1, and 'none' tweets into 0.

In [3]:
def divide_data(dataset):
    data = []
    target = []
    for i in dataset:
        data.append(i[1])
        if i[2] == 'sexism':
            target.append(1)
        elif i[2] == 'racism':
            target.append(1)
        else:
            target.append(0)
    return data, target

I created a class called HateSpeechDetector, which does some simple data cleaning and prep tasks for the strings (gets rid of punctuation in the clean() function, tokenizes the text using regular expressions in the tokenize() function, and gets the word counts in the get_word_counts() function). It then goes into the actual modeling part of the Naive Bayes function. In order to not break up the class, I will add more comments within the code cell rather than in markdown.

In [4]:
class HateSpeechDetector(object):
    #Gets rid of punctuation from the string
    def clean(self, s):
        translator = str.maketrans("", "", string.punctuation)
        return s.translate(translator)
 
    #Converts all text to lowercase and splits the strings into one-word tokens.
    def tokenize(self, text):
        text = self.clean(text).lower()
        return re.split("\W+", text)
 
    #Get the frequency counts of each word
    def get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts
    
    #Fit the model to the training data
    def fit(self, X, Y):
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()
 
        ###Calculate the log class priors###
        '''When I tried getting just the priors without doing a log I got very bad results; I 
        referenced the Stanford NLP website's page on Naive Bayes text classification
        (https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)
        where they explain this is due to floating point underflow - so I instead took the sum 
        of the logs of the probabilitiies rather than the product of the probabilities, 
        as they suggested'''
        n = len(X)
        self.log_class_priors['hatespeech'] = math.log(sum(1 for label in Y if label == 1) / n)
        self.log_class_priors['none'] = math.log(sum(1 for label in Y if label == 0) / n)
        
        #initialize the dictionaries for hatespeech and none
        self.word_counts['hatespeech'] = {}
        self.word_counts['none'] = {}
 
        #count the words for each category 
        for x, y in zip(X, Y):
            c = 'hatespeech' if y == 1 else 'none'
            counts = self.get_word_counts(self.tokenize(x))
            for word, count in counts.items():
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0
 
                self.word_counts[c][word] += count
    
    #predict based on the test set
    def predict(self, X):
        result = []
        for x in X:
            counts = self.get_word_counts(self.tokenize(x))
            
            hatespeech_score = 0
            none_score = 0
            for word, _ in counts.items():
                if word not in self.vocab: continue
            
            # use Laplace smoothing to account for sparsity
                log_w_given_hatespeech = math.log( (self.word_counts['hatespeech'].get(word, 0.0) + 1) / (sum(self.word_counts['hatespeech'].values()) + len(self.vocab)) )
                log_w_given_none = math.log( (self.word_counts['none'].get(word, 0.0) + 1) / (sum(self.word_counts['none'].values()) + len(self.vocab)) )
 
                hatespeech_score += log_w_given_hatespeech
                none_score += log_w_given_none
 
            hatespeech_score += self.log_class_priors['hatespeech']
            none_score += self.log_class_priors['none']
 
            #assign result based on which score is larger
            if hatespeech_score > none_score:
                result.append(1)
            else:
                result.append(0)
        return result

The main function here brings all of these functions together and gets evaluation metrics (accuracy and F1).

In [14]:
def main():
    #load in hatespeech dataset
    ds = loadCsv('hs_for_naive.csv')
    
    random.shuffle(ds)
    
    #divide hatespeech dataset into the X values (the tweet string) and y values (class)
    X, y = divide_data(ds)
    #create HateSpeechDetector object
    MNB = HateSpeechDetector()
    #fit the first 15000 values
    MNB.fit(X[15000:], y[15000:])
    
    #test with the last 100 values of the dataset
    pred = MNB.predict(X[:100])
    true = y[:100]
    
    #calculate accuracy, recall, and precision
    accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
    recall = sum(1 for i in range(len(pred)) if (pred[i] == 1 & true[i] == 1))/(sum(true))
    precision = sum(1 for i in range(len(pred)) if (pred[i] == 1 & true[i] == 1))/(sum(pred))
    
    #calculate F1
    F1 = 2*((recall*precision)/(recall+precision))
    print("Accuracy: " + "{0:.4f}".format(accuracy))
    print("F1: " + "{0:.4f}".format(F1)) 

 main()